In [ ]:
import numpy as np
import scipy.signal as signal
import json
import os

class MicrophoneCalibrator:
    def __init__(self, ref_mic_index=0):
        self.ref_mic_index = ref_mic_index
        self.calibration_data = {}

    def analyze_frequency_response(self, sweep_signal, mic_recordings, sample_rate):
        """
        Analyze the frequency response of each microphone compared to the reference microphone.

        Parameters:
        - sweep_signal: The original frequency sweep signal.
        - mic_recordings: A list of arrays containing recordings from each microphone.
        - sample_rate: Sampling rate of the recordings.
        """
        # Compute the FFT of the reference microphone's response
        ref_response = mic_recordings[self.ref_mic_index]
        ref_fft = np.abs(np.fft.rfft(ref_response))
        
        # Normalize by the sweep signal's FFT
        sweep_fft = np.abs(np.fft.rfft(sweep_signal))
        ref_fft /= sweep_fft

        calibration_factors = []

        for i, mic_response in enumerate(mic_recordings):
            mic_fft = np.abs(np.fft.rfft(mic_response))
            mic_fft /= sweep_fft

            # Calculate calibration factor as ratio of reference to microphone FFT
            cal_factor = ref_fft / mic_fft
            calibration_factors.append(cal_factor)

        # Save frequency bins for applying the calibration later
        frequency_bins = np.fft.rfftfreq(len(sweep_signal), 1 / sample_rate)
        self.calibration_data = {
            "frequency_bins": frequency_bins.tolist(),
            "calibration_factors": [cf.tolist() for cf in calibration_factors]
        }

    def save_calibration(self, filepath):
        """Save the calibration data to a JSON file."""
        with open(filepath, 'w') as f:
            json.dump(self.calibration_data, f)

    def load_calibration(self, filepath):
        """Load calibration data from a JSON file."""
        if os.path.exists(filepath):
            with open(filepath, 'r') as f:
                self.calibration_data = json.load(f)
        else:
            raise FileNotFoundError(f"Calibration file not found: {filepath}")

    def apply_calibration(self, mic_recordings):
        """
        Apply calibration to new recordings.

        Parameters:
        - mic_recordings: A list of arrays containing recordings from each microphone.

        Returns:
        - A list of calibrated microphone recordings.
        """
        if not self.calibration_data:
            raise ValueError("Calibration data is not loaded.")

        frequency_bins = np.array(self.calibration_data["frequency_bins"])
        calibrated_recordings = []

        for i, mic_response in enumerate(mic_recordings):
            mic_fft = np.fft.rfft(mic_response)
            calibration_factor = np.array(self.calibration_data["calibration_factors"][i])
            calibrated_fft = mic_fft * calibration_factor
            calibrated_response = np.fft.irfft(calibrated_fft, n=len(mic_response))
            calibrated_recordings.append(calibrated_response)

        return calibrated_recordings

In [ ]:
# Example usage
if __name__ == "__main__":
    # Load or generate data
    sample_rate = 48000  # 48 kHz
    duration = 5  # 5 seconds
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

    # Generate a frequency sweep signal
    sweep_signal = signal.chirp(t, f0=20, f1=20000, t1=duration, method='logarithmic')

    # Simulate microphone recordings (replace with actual data loading in practice)
    num_mics = 4
    mic_recordings = [sweep_signal + np.random.normal(0, 0.01, len(sweep_signal)) for _ in range(num_mics)]

    # Initialize the calibrator
    calibrator = MicrophoneCalibrator(ref_mic_index=0)

    # Analyze frequency response and save calibration
    calibrator.analyze_frequency_response(sweep_signal, mic_recordings, sample_rate)
    calibrator.save_calibration("calibration_data.json")

    # Load calibration and apply to new recordings
    calibrator.load_calibration("calibration_data.json")
    new_mic_recordings = [sweep_signal + np.random.normal(0, 0.01, len(sweep_signal)) for _ in range(num_mics)]
    calibrated_recordings = calibrator.apply_calibration(new_mic_recordings)

    print("Calibration applied successfully.")